# Load Argo data
This notebook loads Argo Data from the Labrador Sea in between a specified time frame as an xarray dataset and saves it as an nc-file. The data loaded contains profiles of absolute salinity (SA), conservative temperatur (CT) and potential vorticity (PV) as a function of pressure sorted by depth levels and profile number. First the dataset for the whole period (2002-2023) is created, secondly a subset with different end and start dates is created. This subset uses the time span used in Holte & Straneo (2017) and was used for comparison.

In [4]:
# install argopy libary from github
!pip install git+https://github.com/euroargodev/argopy.git@master

^C


In [19]:
#--------------- import libaries ---------------------------------------------
import xarray as xr
import argopy
argopy.set_options(mode='research') # to only allow for qualitifly high data
from argopy import DataFetcher as ArgoDataFetcher


#--------------- SET VARIABLES -----------------------------------------------
# Set time range
start_year = 2002
end_year   = 2023

# Coordinates of Labrador Sea 
Lon0 = -66
Lon1 = -44
Lat0 = 45
Lat1 = 68
# maximum water depth
max_depth = 2000

# set file name
filename = 'LabSea_Argo_2002_2023.nc'
datapath = '.\\data\\'
#-----------------------------------------------------------------------------

# Create an empty xarray dataset to store the data
ag = None

# Loop through each year
for year in range(start_year, end_year+1):

    # Set the time range for the current year
    year_start = f'{year}-01-01'
    year_end = f'{year}-12-31'

    # Load Argo data for the current year and region
    fetcher = ArgoDataFetcher(mode='research', timeout=100000)
    print(year)
    ds_year = fetcher.region([Lon0, Lon1, Lat0, Lat1, 0, max_depth, year_start, year_end]).to_xarray()
    
    # Sort data into profiles, select variables and concatenate the current year's data to the main dataset
    if ag is None:
        ag = ds_year.argo.point2profile()
        ag.argo.teos10(['SA', 'CT', 'PV'])
    else:
        ag_points = ds_year.argo.point2profile()
        ag_points.argo.teos10(['SA', 'CT', 'PV'])
        ag = xr.concat([ag, ag_points], dim='N_PROF')

# Print the final dataset to check variables
print(ag)

#save dataset as netcdf file
ag.to_netcdf(datapath + filename)

2016
2017


FSTimeoutError: 

# Subdataset

In [ ]:
#--------------- SET VARIABLES -----------------------------------------------
# Set time range
# set time to only load dataset for reproducing Holte & Straneo (2017) 
# timespan set to '2002-03-01' to '2016-04-30' below
start_year = 2002
end_year   = 2016

# Coordinates of Labrador Sea 
Lon0 = -66
Lon1 = -44
Lat0 = 45
Lat1 = 68
# maximum water depth
max_depth = 2000

# set file name
filename2 = 'LabSea_Argo_2002_2016.nc'
datapath = '.\\data\\'
#-----------------------------------------------------------------------------

# Create an empty array
ag2 = None

for year in range(start_year2, end_year2+1):

    # Set the time range for the current year
    if (start_year2 < year < end_year2):
        year_start = f'{year}-01-01'
        year_end = f'{year}-12-31'
    elif year == 2002:
        year_start = f'{year}-03-01' # time span for reproducing Holte & Straneo (2017) data
        year_end = f'{year}-12-31'
    elif year == 2016:
        year_start = f'{year}-01-01'
        year_end = f'{year}-04-30' # time span for reproducing Holte & Straneo (2017) data

    # Load Argo data for the current year and region
    fetcher = ArgoDataFetcher(mode='research', timeout=100000)
    print(year)
    ds_year = fetcher.region([Lon0, Lon1, Lat0, Lat1, 0, max_depth, year_start, year_end]).to_xarray()
    
    # Sort data into profiles, select variables and concatenate the current year's data to the main dataset
    if ag2 is None:
        ag2 = ds_year.argo.point2profile()
        ag2.argo.teos10(['SA', 'CT', 'PV'])
    else:
        ag_points = ds_year.argo.point2profile()
        ag_points.argo.teos10(['SA', 'CT', 'PV'])
        ag2 = xr.concat([ag2, ag_points], dim='N_PROF')

# Print the final dataset to check variables
print(ag2)

#save dataset as netcdf file
ag2.to_netcdf(datapath + filename2)